# Training

In [1]:
import torch
import torchvision.transforms as T
import torch.nn as nn
import torch.nn.functional as F
from train_test.train import Trainer
import torchvision.models as models

from utility.get_data import S3ImageFolder

2024-08-02 20:23:15.340743: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
imagenet_a_path = "imagenet-a"
imagenet_b_path = "imagenetv2-matched-frequency-format-val/"

In [3]:
def get_data(batch_size, img_root, seed, transform = None):

    # Load data
    data = S3ImageFolder(root=img_root, transform=transform)
    
    # Create train and test splits (80/20)
    num_samples = len(data)
    training_samples = int(num_samples * 0.8 + 1)
    val_samples = int(num_samples * 0.1)
    test_samples = num_samples - training_samples - val_samples

    torch.manual_seed(seed)
    training_data, val_data, test_data = torch.utils.data.random_split(data, [training_samples, val_samples, test_samples])
    
    # Initialize dataloaders
    train_loader = torch.utils.data.DataLoader(training_data, batch_size, shuffle=True, num_workers=4)
    val_loader = torch.utils.data.DataLoader(val_data, batch_size, shuffle=False, num_workers=4)
    test_loader = torch.utils.data.DataLoader(test_data, batch_size, shuffle=False, num_workers=4)

    return train_loader, val_loader, test_loader

In [4]:
transform = T.Compose([
        T.Resize((256, 256)),
        T.RandomCrop((224, 224)),
        T.ToTensor(),
        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

train_loader, val_loader, test_loader = get_data(batch_size=32, 
                                                 img_root=imagenet_a_path, 
                                                 transform = transform, 
                                                 seed=42)

In [5]:
data_loaders = {
    "train_loader": train_loader,
    "val_loader": val_loader,
    "test_loader": test_loader,
}     

In [3]:
model = models.resnet50(pretrained=True)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 1000)

for param in model.parameters():
    param.requires_grad = False

for param in model.fc.parameters():
    param.requires_grad = True

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=0.005)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [9]:
# checkpoint = torch.load("/home/sagemaker-user/Domain-Shift-Computer-Vision/experiments/Resnet50_ImagenetA_SGD/checkpoint.pth")
# model.load_state_dict(checkpoint["model"])
# optimizer.load_state_dict(checkpoint["optimizer"])

In [9]:
model_trainer = Trainer(data_loaders=data_loaders,
                         dataset_name="Imagenet-A",
                         model=model,
                         optimizer=optimizer,
                         loss_fn=nn.CrossEntropyLoss(),
                         device=device,
                         seed=42,
                         exp_path="/home/sagemaker-user/Domain-Shift-Computer-Vision/experiments",
                         exp_name="Resnet50_ImagenetA_SGD_1",
                         use_early_stopping=True,
                         trained = False
                       )

In [ ]:
# Resnet50: 6 epochs, early stopping
for i in range(2):
    model_trainer.main(epochs=3,
                       log_interval=0.01)

In [ ]:
# visualize with tensorboard
%reload_ext tensorboard
%tensorboard --logdir=/home/sagemaker-user/Domain-Shift-Computer-Vision/experiments # experiment path

-----

# MEMO

In [1]:
import torch
import torchvision.transforms as T
import torch.nn as nn
import torch.nn.functional as F
from train_test.train import Trainer
import torchvision.models as models
import torchvision

from utility.get_data import S3ImageFolder

2024-08-02 23:13:11.783886: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from MEMO.MEMO import MEMO

In [3]:
imagenet_a_path = "imagenet-a"
imagenet_b_path = "imagenetv2-matched-frequency-format-val/"

In [4]:
checkpoint_path = "/home/sagemaker-user/Domain-Shift-Computer-Vision/experiments/Resnet50_ImagenetA_SGD/checkpoint.pth"

In [5]:
fine_tuned_model = models.resnet50(weights="IMAGENET1K_V1")
fine_tuned_optimizer = torch.optim.SGD(fine_tuned_model.parameters(), lr=0.1)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [6]:
import torchvision.transforms as T

# List of augmentations
augmentations = [
    T.RandomHorizontalFlip(p=1),
    T.RandomVerticalFlip(p=1),
    T.RandomRotation(degrees=30),
    T.RandomRotation(degrees=60),
    T.ColorJitter(brightness=0.2),
    T.ColorJitter(contrast=0.2),
    T.ColorJitter(saturation=0.2),
    T.ColorJitter(hue=0.2),
    T.RandomAffine(degrees=0, translate=(0.1, 0.1)),
    T.RandomRotation(degrees=15),
    T.RandomAdjustSharpness(sharpness_factor=2, p=1),
    T.RandomGrayscale(p=1),
    T.RandomInvert(p=1),
    T.RandomAutocontrast(p=1),
    T.GaussianBlur(kernel_size=5),
]

In [7]:
MEMO_resnet50 = MEMO(
    model = models.resnet50,
    optimizer = torch.optim.SGD,  
    fine_tuned_model = fine_tuned_model, 
    fine_tuned_optimizer = fine_tuned_optimizer, 
    checkpoint_path = checkpoint_path, 
    device = device
)

In [8]:
lr_setting = {
    "classifier" : [["fc.weight","fc.bias"], 0.001],
    "other_params" : 0.0001
}

In [9]:
MEMO_resnet50.test_MEMO(
     augmentations = augmentations, 
     num_augmentations = 12,
     seed_augmentations = 42,
     seed_data = 42, 
     batch_size = 32, 
     img_root = imagenet_a_path,
     MEMO = True,
     fine_tuned = True,
     lr_setting = None,
     top_k = 6
)

/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.